In [1]:
from helpers import sample_data, load_data, standardize
import numpy as np


# load data.
height, weight, gender = load_data()

# build sampled x and y.
seed = 1
y = np.expand_dims(gender, axis=1)
X = np.c_[height.reshape(-1), weight.reshape(-1)]
y, X = sample_data(y, X, seed, size_samples=200)
x, mean_x, std_x = standardize(X)

In [10]:

# General Functions 

def sigmoid(t):
    """apply sigmoid function on t.

    Args:
        t: scalar or numpy array

    Returns:
        scalar or numpy array

    >>> sigmoid(np.array([0.1]))
    array([0.52497919])
    >>> sigmoid(np.array([0.1, 0.1]))
    array([0.52497919, 0.52497919])
    """
    
    return np.e**t/(1+np.e**t)


def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)

    Returns:
        a non-negative loss

    >>> y = np.c_[[0., 1.]]
    >>> tx = np.arange(4).reshape(2, 2)
    >>> w = np.c_[[2., 3.]]
    >>> round(calculate_loss(y, tx, w), 8)
    1.52429481
    """
    if y.shape[0] != tx.shape[0]:
        raise ValueError("Mismatch: The number of samples in y and tx must be the same.")
    if tx.shape[1] != w.shape[0]:
        raise ValueError("Mismatch: The number of features in tx must match the number of weights in w.")
    
    # Calculate log-loss L        
    sig_pred = sigmoid(tx @ w)
    loss = -1 / len(y) * (y.T @ np.log(sig_pred) + (1 - y).T @ np.log(1 - sig_pred)).item()
    
    return loss


In [3]:
def calculate_gradient(y, tx, w):
    """compute the gradient of loss.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)

    Returns:
        a vector of shape (D, 1)

    >>> np.set_printoptions(8)
    >>> y = np.c_[[0., 1.]]
    >>> tx = np.arange(6).reshape(2, 3)
    >>> w = np.array([[0.1], [0.2], [0.3]])
    >>> calculate_gradient(y, tx, w)
    array([[-0.10370763],
           [ 0.2067104 ],
           [ 0.51712843]])
    """
    
    return 1 / len(y) * tx.T @ (sigmoid(tx @ w) - y)



In [4]:


def calculate_hessian(y, tx, w):
    """return the Hessian of the loss function.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)

    Returns:
        a hessian matrix of shape=(D, D)

    >>> y = np.c_[[0., 1.]]
    >>> tx = np.arange(6).reshape(2, 3)
    >>> w = np.array([[0.1], [0.2], [0.3]])
    >>> calculate_hessian(y, tx, w)
    array([[0.28961235, 0.3861498 , 0.48268724],
           [0.3861498 , 0.62182124, 0.85749269],
           [0.48268724, 0.85749269, 1.23229813]])
    """
    # Creating S matrix with sigma values in diagonale
    sigmas = []
    for xi in tx:
        sig_xi = sigmoid(xi.T @ w)
        temp = sig_xi * (1 - sig_xi)
        sigmas.append(temp.item())
    
    # Matrix with sig (1- sig) in diagonale
    S = np.diag(sigmas)
    
    return 1 / len(y) * tx.T @ S @ tx



In [5]:
def learning_by_newton_method(y, tx, w, gamma):
    """
    Do one step of Newton's method.
    Return the loss and updated w.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)
        gamma: scalar

    Returns:
        loss: scalar number
        w: shape=(D, 1)

    >>> y = np.c_[[0., 0., 1., 1.]]
    >>> np.random.seed(0)
    >>> tx = np.random.rand(4, 3)
    >>> w = np.array([[0.1], [0.5], [0.5]])
    >>> gamma = 0.1
    >>> loss, w = learning_by_newton_method(y, tx, w, gamma)
    >>> round(loss, 8)
    0.71692036
    >>> w
    array([[-1.31876014],
           [ 1.0590277 ],
           [ 0.80091466]])
    """
    
    # Calculating loss, gradient and hessian
    loss = calculate_loss(y, tx, w)
    gradient = calculate_gradient(y, tx, w)
    hessian = calculate_hessian(y, tx, w)
    
    # Calculate w_t+1 using Newtons method
    w -= gamma * np.linalg.inv(hessian) @ gradient
    
    return loss, w






In [6]:

def logistic_regression(y, tx, initial_w, max_iter, gamma, threshold = 1e-8):
    """
    Do logistic regression until the threshold or max_iter is reached.
    
    Args:
        y:            shape=(N, 1)
        tx:           shape=(N, D+1)
        initial_w:    shape=(D+1, 1)
        max_iter:     int
        gamma:        float
        
    Returns: 
        w:         shape=(D+1, 1)
        losses:    array with losses throughout iteration
        
    Example:
    w_final, losses = logistic_regression(y, tx, initial_w, max_iter, gamma)
    
    
    """
    losses = []
    w = initial_w

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, tx, w, gamma)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        losses.append(loss)
        
        # converge criterion
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
            
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    
    return w, losses

In [12]:
tx = np.c_[np.ones((y.shape[0], 1)), x]
w = np.zeros((tx.shape[1], 1))
w, losses = logistic_regression(y, tx, w, 10000, 0.005)

Current iteration=0, loss=0.6931471805599453
Current iteration=100, loss=0.4737088075802126
Current iteration=200, loss=0.35973183949541865
Current iteration=300, loss=0.2925462485506452
Current iteration=400, loss=0.2522680345421841
Current iteration=500, loss=0.22890719580979738
Current iteration=600, loss=0.21616297884578786
Current iteration=700, loss=0.20971388852369552
Current iteration=800, loss=0.2066923330386357
Current iteration=900, loss=0.20537129932725495
Current iteration=1000, loss=0.20482503658681853
Current iteration=1100, loss=0.20460824361518343
Current iteration=1200, loss=0.20452461223715226
Current iteration=1300, loss=0.2044929476297321
Current iteration=1400, loss=0.20448110107540995
Current iteration=1500, loss=0.20447670199007084
loss=0.2044751048336365
